# Based on:
# Beautiful geo plot with heatmap & date-scroll

https://www.jumpingrivers.com/blog/interactive-maps-python-covid-19-spread/

In [50]:
# %conda install -c conda-forge branca

In [51]:
# %conda install -c conda-forge folium

In [52]:
import branca.colormap as cm
import folium
import geopandas as gpd
import numpy as np
import pandas as pd

# Change format of Nigeria state map so folium can use it

In [53]:
import fiona
import geopandas as gpd

In [54]:
states_file = "nga_admbnda_osgof_eha_itos.gdb/nga_admbnda_osgof_eha_itos.gdb/nga_admbnda_osgof_eha_itos.gdb/nga_admbnda_osgof_eha_itos.gdb/"

layers = fiona.listlayers(states_file)
zones_gdf = gpd.read_file(states_file, layer=1)
zones_gdf.crs = "EPSG:4326"

zones_gdf = zones_gdf.set_index('admin1Name_en')
zones_gdf = zones_gdf[['geometry']]
zones_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 37 entries, Abia to Zamfara
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  37 non-null     geometry
dtypes: geometry(1)
memory usage: 592.0+ bytes


# Add in my Nigerian Covid data

In [55]:
# get my Nigerian data

covid_states = pd.read_csv('giant_df_trimmed.csv')
# covid_states.head()

In [56]:
# We also need to convert the ObservationDate to unix time in nanoseconds:

covid_states['date_sec'] = pd.to_datetime(covid_states['date']).astype(int) / 10**9
covid_states['date_sec'] = covid_states['date_sec'].astype(int).astype(str)
covid_states.drop('date', axis=1)

,Unnamed: 0,admin1Name_en,total_confirmed,new_confirmed,total_discharged,new_dicharged,total_deaths,new_deaths,total_active,date_sec
0,0,Abia,2,0,0,0,0,0,2,1588377600
1,1,Abia,2,0,1,1,0,0,1,1588464000
2,2,Abia,2,0,1,0,0,0,1,1588550400
3,3,Abia,2,0,1,0,0,0,1,1588636800
4,4,Abia,2,0,1,0,0,0,1,1588723200
...,...,...,...,...,...,...,...,...,...,...
3722,3722,Zamfara,77,0,71,0,5,0,1,1597363200
3723,3723,Zamfara,77,0,71,0,5,0,1,1597449600
3724,3724,Zamfara,77,0,71,0,5,0,1,1597536000
3725,3725,Zamfara,77,0,71,0,5,0,1,1597622400


In [57]:
# We can now select the columns needed for the map and discard the others:

covid_states.set_index('admin1Name_en', inplace=True, drop=False)
covid_states = covid_states.rename(columns={'admin1Name_en': 'state'})

# join geo & case data
covid_states = zones_gdf.join(covid_states, how='outer')

# clean up
covid_states = covid_states[['state', 'geometry', 'date_sec', 'total_confirmed', 'new_confirmed',
                             'total_discharged', 'new_dicharged', 'total_deaths', 'new_deaths', 'total_active']]

covid_states.head()

,state,geometry,date_sec,total_confirmed,new_confirmed,total_discharged,new_dicharged,total_deaths,new_deaths,total_active
admin1Name_en,,,,,,,,,,
Abia,Abia,"MULTIPOLYGON (((7.38681 6.03667, 7.38729 6.036...",1588377600,2,0,0,0,0,0,2
Abia,Abia,"MULTIPOLYGON (((7.38681 6.03667, 7.38729 6.036...",1588464000,2,0,1,1,0,0,1
Abia,Abia,"MULTIPOLYGON (((7.38681 6.03667, 7.38729 6.036...",1588550400,2,0,1,0,0,0,1
Abia,Abia,"MULTIPOLYGON (((7.38681 6.03667, 7.38729 6.036...",1588636800,2,0,1,0,0,0,1
Abia,Abia,"MULTIPOLYGON (((7.38681 6.03667, 7.38729 6.036...",1588723200,2,0,1,0,0,0,1


In [58]:
# start plotting etc

In [59]:
# We have to first initialise the map.
# Folium allows the use of different map tiles.
# If we do not specify a map, it defaults to OpenStreetMap.
# Here, we will use 'cartodbpositron':

mymap = folium.Map(tiles='cartodbpositron')

# the continents are continually repeated and the map can be panned endlessly from either side.
# In order to prevent this from happening, we set a minimum zoom and set max_bounds=True:

mymap_fix_boundary = folium.Map(min_zoom=2, max_bounds=True, tiles='cartodbpositron')

In [60]:
# Now we define a colour map in terms of new_confirmed cases

max_colour = max(covid_states['new_confirmed'])
min_colour = min(covid_states['new_confirmed'])
cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
covid_states['colour'] = covid_states['new_confirmed'].map(cmap)

In [61]:
# Next, we construct our style dictionary

state_list = covid_states['state'].unique().tolist()
state_idx = range(len(state_list))

style_dict = {}
for i in state_idx:
    state = state_list[i]
    result = covid_states[covid_states['state'] == state]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['date_sec']] = {'color': r['colour'], 'opacity': 0.7}
    style_dict[str(i)] = inner_dict

In [62]:
# Then we need to make a dataframe containing the features for each country:

states_df = covid_states[['geometry']]
states_gdf = gpd.GeoDataFrame(states_df)
states_gdf = states_gdf.drop_duplicates().reset_index()

In [63]:
# Finally, we create our map and add a colourbar:

# The TimeSliderChoropleth class needs at least two arguments:
# a GeoJSON file containing the features (in this case, the countries) and a style dictionary.

from folium.plugins import TimeSliderChoropleth

slider_map = folium.Map(min_zoom=2, max_bounds=True,tiles='cartodbpositron')


# convert Nigeria states map into json (via crs)
# folium likes json => good
# states_json = zones_gdf.to_crs(epsg='4326').to_json()

# make a map
# points = folium.features.GeoJson(states_json)
# slider_map.add_child(points)


_ = TimeSliderChoropleth(
    data = states_gdf.to_json(),
    styledict = style_dict,
).add_to(slider_map)

# add legend, ie colour scale
_ = cmap.add_to(slider_map)
cmap.caption = "New confirmed cases"


slider_map
# slider_map.save(outfile='TimeSliderChoropleth.html')